In [ ]:
class Solution:

    def minNumberOfSemesters(self, n: int, relations: List[List[int]], k: int) -> int:
        pre = [0] * n  # 前驱
        for x, y in relations:
            pre[y - 1] |= 1 << (x - 1)
        target = (1 << n) - 1  # 目标全完成

        @cache
        def dfs(sta):  # 完成sta课最少需要几学期 意味着sta的补集都完成
            if sta == 0:
                return 0
            csta = target ^ sta  # target & ~sta # 补集 sta就是target子集 可以直接target ^ sta
            # 检查实际可以先完成的课
            mask = 0
            for i, x in enumerate(pre):
                if sta >> i & 1 and x & csta == x:
                    mask |= 1 << i
            if mask.bit_count() <= k:  # 可以正常全选完成
                return dfs(sta ^ mask) + 1
            # 找小于k的集合先完成
            ans = n
            sub = mask  # 枚举子集 见 https://leetcode.cn/circle/discuss/CaOJ45/
            while sub:
                if sub.bit_count() == k: # 不用 <= 多学一点是一点
                    ans = min(ans, dfs(sta ^ sub) + 1)
                sub = (sub - 1) & mask
            return ans

        return dfs(target)

        # f = [n] * (target + 1)
        # f[0] = 0
        # for sta in range(1, target + 1):
        #     csta = target ^ sta  # target & ~sta # 补集 sta就是target子集 可以直接target ^ sta
        #     # 检查实际可以先完成的课
        #     mask = 0
        #     for i, x in enumerate(pre):
        #         if sta >> i & 1 and x & csta == x:
        #             mask |= 1 << i
        #     if mask.bit_count() <= k:  # 可以正常全选完成
        #         f[sta] = f[sta ^ mask] + 1
        #     else:
        #         # 找小于k的集合先完成
        #         sub = mask  # 枚举子集 见 https://leetcode.cn/circle/discuss/CaOJ45/
        #         while sub:
        #             if sub.bit_count() == k:
        #                 f[sta] = min(f[sta], f[sta ^ sub] + 1)
        #             sub = (sub - 1) & mask
        # return f[target]

In [ ]:
class Solution:

    def minNumberOfSemesters(self, n: int, relations: List[List[int]], k: int) -> int:
        pre = [0] * n  # 前驱
        for x, y in relations:
            pre[y - 1] |= 1 << (x - 1)
        target = (1 << n) - 1  # 目标全完成
        step = 0
        q = deque([0])
        vis = [0] * (1 << n)  # 2 ** n 每个位置0或1
        vis[0] = 1
        while q:
            for _ in range(len(q)):
                sta = q.popleft()
                canstudy = []
                for i in range(n):
                    # 该课没学过 且 前驱都学过了  是当前状态子集
                    if pre[i] | sta == sta and sta >> i & 1 == 0:
                        canstudy.append(i)
                for ys in combinations(canstudy, min(len(canstudy), k)):
                    nsta = sta
                    for y in ys:
                        nsta |= 1 << y
                    if nsta == target: return step + 1
                    if vis[nsta]: continue
                    vis[nsta] = 1
                    q.append(nsta)
            step += 1


In [ ]:
from itertools import combinations


class Solution:

    def minNumberOfSemesters(self, n: int, relations: List[List[int]], k: int) -> int:
        pre = [0] * n  # 前驱
        for x, y in relations:
            pre[y - 1] |= 1 << (x - 1)
        target = (1 << n) - 1  # 目标全完成

        @lru_cache(None)
        def dfs(sta):  # 从state到target最少步数
            if sta == target:
                return 0
            canstudy = [i for i in range(n) if pre[i] | sta == sta and sta >> i & 1 == 0]
            ans = n
            for ys in combinations(canstudy, min(len(canstudy), k)):
                nsta = sta
                for y in ys:
                    nsta |= 1 << y
                ans = min(ans, dfs(nsta) + 1)
            return ans

        return dfs(0)

        # f = [n] * (target + 1)
        # f[target] = 0
        # for sta in range(target, -1, -1):
        #     canstudy = [i for i in range(n) if pre[i] | sta == sta and sta >> i & 1 == 0]
        #     for ys in combinations(canstudy, min(len(canstudy), k)):
        #         nsta = sta
        #         for y in ys:
        #             nsta |= 1 << y
        #         f[sta] = min(f[sta], f[nsta] + 1)
        # return f[0]